In [7]:
using Pkg
Pkg.add("Distances")
using JuMP,Gurobi,Distances

  Resolving package versions...
  Installed Distances ─ v0.9.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [b4f34e82] + Distances v0.9.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [b4f34e82] + Distances v0.9.0
┌ Info: Precompiling Distances [b4f34e82-e78d-54a5-968a-f98e89d6e8f7]
└ @ Base loading.jl:1278


In [8]:
using CSV
data = CSV.read("data_transfered.csv")

,ID,Country or area,Country code,Capital City
,Int64,String,Int64,String
1,1,Afghanistan,4,Kabul
2,2,Albania,8,Tiran\x91 (Tirana)
3,3,Algeria,12,El Djaza\x95r (Algiers)
4,4,American Samoa,16,Pago Pago
5,5,Andorra,20,Andorra la Vella
6,6,Angola,24,Luanda
7,7,Anguilla,660,The Valley
8,8,Antigua and Barbuda,28,St. John's
9,9,Argentina,32,Buenos Aires


In [9]:
I = [1:241;]
J = [1:241;]
de = 110.25

110.25

In [10]:
c = zeros((241,241))
for i = 1:241
    for j = 1:241
        c[i,j] = de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
    end
end

In [13]:
cc = zeros((241,241))
for i = 1:241
    for j = 1:241
        cc[i,j] = haversine((data[i,8],data[i,9]),(data[j,8],data[j,9]),6372.8)
    end
end

In [15]:
cc[1,2]

5509.149352017856

In [16]:
c[1,2]

4867.222080544683

In [5]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 5
Model_p5 = Model(Gurobi.Optimizer)
@variable(Model_p5,x[i in I],Bin) # if city i is server
@variable(Model_p5,y[i in I,j in I],Bin) #if city i is served by city j
@objective(Model_p5,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in I))
@constraints(Model_p5,begin
        [i in I], sum(y[i,j] for j in I) == 1
        [i in I , j in I], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [6]:
optimize!(Model_p5)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0x79fa487c
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.39s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 1.204405e+09

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 3.702516e+08, 2387 iterations, 0.29 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [7]:
println("p = 5")
print("Optimal value: ")
println(objective_value(Model_p5))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 5
Optimal value: 3.702515838119901e8
node 9 80 88 180 190 are selected as distribution centers

In [8]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 10
Model_p10 = Model(Gurobi.Optimizer)
@variable(Model_p10,x[i in I],Bin) # if city i is server
@variable(Model_p10,y[i in I,j in J],Bin) #if city i is served by city j
@objective(Model_p10,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_p10,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [9]:
optimize!(Model_p10)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0x0f2ba2e3
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.38s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 8.135880e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 2.562010e+08, 1897 iterations, 0.28 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [10]:
println("p = 10")
print("Optimal value: ")
println(objective_value(Model_p10))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 10
Optimal value: 2.56201002868491e8
node 9 51 80 88 103 161 167 180 190 208 are selected as distribution centers

In [11]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 15
Model_p15 = Model(Gurobi.Optimizer)
@variable(Model_p15,x[i in I],Bin) # if city i is server
@variable(Model_p15,y[i in I,j in J],Bin) #if city i is served by city j
@objective(Model_p15,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_p15,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [12]:
optimize!(Model_p15)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0x807d1875
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.39s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 7.822088e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.928723e+08, 1518 iterations, 0.26 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [13]:
println("p = 15")
print("Optimal value: ")
println(objective_value(Model_p15))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 15
Optimal value: 1.928723109912253e8
node 9 51 70 80 88 103 126 143 161 167 180 190 192 201 208 are selected as distribution centers

In [14]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 20
Model_p20 = Model(Gurobi.Optimizer)
@variable(Model_p20,x[i in I],Bin) # if city i is server
@variable(Model_p20,y[i in I,j in J],Bin) #if city i is served by city j
@objective(Model_p20,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_p20,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [15]:
optimize!(Model_p20)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0x84fd5e46
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.39s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 5.589340e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.476201e+08, 1453 iterations, 0.26 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [16]:
println("p = 20")
print("Optimal value: ")
println(objective_value(Model_p20))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 20
Optimal value: 1.476201035487954e8
node 9 46 47 51 64 70 80 88 103 104 112 122 126 129 143 161 167 180 192 201 are selected as distribution centers

In [17]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 25
Model_p25 = Model(Gurobi.Optimizer)
@variable(Model_p25,x[i in I],Bin) # if city i is server
@variable(Model_p25,y[i in I,j in J],Bin) #if city i is served by city j
@objective(Model_p25,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_p25,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [18]:
optimize!(Model_p25)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0xcbec2eac
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.39s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 4.796093e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.188004e+08, 1265 iterations, 0.24 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [19]:
println("p = 25")
print("Optimal value: ")
println(objective_value(Model_p25))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 25
Optimal value: 1.1880040562413189e8
node 9 30 46 47 51 64 70 80 88 103 104 112 122 126 129 139 143 161 167 168 177 180 192 201 214 are selected as distribution centers

In [20]:
#de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
p = 30
Model_p30 = Model(Gurobi.Optimizer)
@variable(Model_p30,x[i in I],Bin) # if city i is server
@variable(Model_p30,y[i in I,j in J],Bin) #if city i is served by city j
@objective(Model_p30,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_p30,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        #[i in I], y[i,i] == x[i]
        sum(x[i] for i in I) == p
    end)

Academic license - for non-commercial use only


In [21]:
optimize!(Model_p30)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58323 rows, 58322 columns and 174484 nonzeros
Model fingerprint: 0xc53e2800
Variable types: 0 continuous, 58322 integer (58322 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.38s
Presolved: 58323 rows, 58322 columns, 174484 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 4.375687e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 9.824545e+07, 1111 iterations, 0.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [22]:
println("p = 30")
print("Optimal value: ")
println(objective_value(Model_p30))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

p = 30
Optimal value: 9.824545331191424e7
node 6 9 17 30 46 47 51 56 64 70 80 88 103 104 112 115 126 129 139 143 161 167 168 177 180 192 199 201 208 214 are selected as distribution centers